# Vector representations of words and documents

This homework is significantly adapted by Lelia Glass from one developed at UPenn by Daphne Ippolito, Anne Cocos, Stephen Mayhew, and Chris Callison-Burch.  

An important idea throughout this assignment (and in computer science in general these days) is VECTORIZATION: the idea of performing operations on entire matrices all at once, rather than writing "for loops" for each individual element.  Even when you understand how to calculate each individual element on its own, "vectorizing" the computation can be a bit tricky, so don't worry if you don't understand it all yet.  This is a learning process!

I also want to point out that even if you don't know much programming, you should be able to follow along in this assignment, which asks you to READ and UNDERSTAND code more than writing it yourself.  Just try to understand as much as you can, and you'll move forward in your learning process, even if you do not understand it all yet.  That's okay!

The assignment is based on Chapter 6 of Jurafsky and Martin (3rd Edition).  Please read that chapter to fully understand what's happening here.  You can also check out these slides: https://web.stanford.edu/~jurafsky/li15/lec3.vector.pdf 

In [23]:
import os
import csv
import subprocess
import re
import random
import numpy as np
import pandas as pd


Let's check out how this function works.

## Term-by-document matrix

First, we write code to create a term-by-document matrix.

In [24]:

def create_term_document_matrix(line_tuples, document_names, vocab):
  '''Returns a numpy array containing the term document matrix for the input lines.
  Inputs:
    line_tuples: A list of tuples, containing the name of the document and 
    a tokenized line from that document.
    document_names: A list of the document names
    vocab: A list of the tokens in the vocabulary
  Let m = len(vocab) and n = len(document_names).
  Returns:
    td_matrix: A mxn numpy array where the number of rows is the number of words
        and each column corresponds to a document. A_ij contains the
        frequency with which word i occurs in document j.
  '''
  vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
  docname_to_id = dict(zip(document_names, range(0, len(document_names))))
  rows = len(vocab)
  print("number of words =" + str(rows))
  cols = len(document_names)
  print("number of documents = " + str(cols))
  td_matrix = np.zeros(shape=(rows, cols))
  for tuple in line_tuples:
        doc_name = tuple[0]
        doc_id = docname_to_id[doc_name]
        words = tuple[1]
        for word in words:
            vocab_id = vocab_to_id[word]
            td_matrix[vocab_id, doc_id] += 1

  return td_matrix



## Demo (and debugging) on a tiny corpus

We illustrate with a tiny, manageable, understandable corpus.

In [41]:
line_tuples_demo = (("cheese_and_crackers", 
                ["cheese", "and", "crackers"]), 
               ("wine_and_cheese", 
                ["wine", "and", "cheese"]), 
            
        )

vocab_demo = set()
doc_names_demo = []

def create_demo(line_tuples_demo):
    for tup in line_tuples_demo:
        doc_name = tup[0]
        doc_names_demo.append(doc_name)
        words = tup[1]
        for word in words:
            vocab_demo.add(word)
    return list(vocab_demo), doc_names_demo

vocab_demo, doc_names_demo = create_demo(line_tuples_demo)

print("##Line Tuples")
print(line_tuples_demo)
print("##Doc Names")
print(doc_names_demo)
print("##Vocab")
print(vocab_demo)

##Line Tuples
(('cheese_and_crackers', ['cheese', 'and', 'crackers']), ('wine_and_cheese', ['wine', 'and', 'cheese']))
##Doc Names
['cheese_and_crackers', 'wine_and_cheese']
##Vocab
['wine', 'crackers', 'and', 'cheese']


Now we're going to create a term-document matrix just on this small example, to see exactly how it works.

In [26]:
print(create_term_document_matrix(line_tuples_demo, doc_names_demo, vocab_demo))

number of words =4
number of documents = 2
[[0. 1.]
 [1. 0.]
 [1. 1.]
 [1. 1.]]


## Question 1

What does this matrix represent?  (Please choose an element of the matrix, indicate its row/column, and explain its meaning.) 

## Your answer to Question 1

This matrix represents the similarity of such documents to each other.

Here, rows represent the words, columns represent the documents.

For example, the first row \[0. 1.\] in the matrix means the numbers of occurance for "wine" in the document "cheese_and_crackers" and "wine_and_cheese". 0 means that "wine" doesn't occur in the document "cheese_and_crackers". 1 means that "wine" appears once in the document "wine_and_cheese".

## Term-context matrix

Now we are creating a term-context matrix.  You can change the size of the context window as an argument to the "create_term_context_matrix" function.

In [10]:
def create_term_context_matrix(tuples, vocab, context_window_size=1):
  '''Returns a numpy array containing the term context matrix for the input lines.
  Inputs:
    line_tuples: A list of tuples, containing the name of the document and 
    a tokenized line from that document.
    vocab: A list of the tokens in the vocabulary
  Let n = len(vocab).
  Returns:
    tc_matrix: A nxn numpy array where A_ij contains the frequency with which
        word j was found within context_window_size to the left or right of
        word i in any sentence in the tuples.
  '''
  vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
  tc_matrix = np.zeros(shape=(len(vocab), len(vocab)))
  for tuple in tuples:
        words = tuple[1]
       #print("###### NEW LINE")
       # print(words)
        # un-comment these to see what's happening!
        
        for i in range(0, len(words)):
            target_word = str(words[i])
            target_word_id = vocab_to_id[target_word]
            left_context = words[i-context_window_size:i]
            right_context = words[i+1:i+context_window_size + 1]
            
            #print("### Target Word")
            #print(target_word)
            #print("Left Context")
            #print(left_context)
            #print("Right Context")
            #print(right_context)
            # you can un-comment these to see what's going on!  Also, try different "Context window" sizes.
            
            for context_word in left_context + right_context:
                context_word_id = vocab_to_id[context_word]
                tc_matrix[target_word_id, context_word_id] += 1
            i += 1

 # print("Word IDs") -- you can un-comment this if you want!
  #for key in vocab_to_id.keys():
   #     print(key + ": " + str(vocab_to_id[key]))
  return tc_matrix



Let's try it out.

In [11]:
print(create_term_context_matrix(line_tuples_demo, vocab_demo, 1))

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 1. 0. 2.]
 [0. 0. 2. 0.]]


## Question 2

What does this matrix represent?  Please choose a single cell of the matrix, indicate the row/column of this cell, and then explain its meaning.  (How do these numbers relate to "cheese", "crackers", "and", "wine")?


## Your answer to Question 2

This matrix counts the occurance of word in the certain context with window size = 1. It can represent the similarity and relatedness of the words in the context.

For example, a cell of matrix\[3, 2\] which has the value 2. Here, the row means the word "cheese" and the column means the word "and". It means that both "cheese and" and "and cheese" appear in the contexts. The word "cheese" and "and" have more closer relatedness or similarity.

## Question 3

Please change the "context window" to 2, and re-run the term-context matrix code.  What happens and why?  Again, please choose a single cell of the matrix and explain its meaning.

In [12]:
print(create_term_context_matrix(line_tuples_demo, vocab_demo, 2))

[[0. 0. 1. 1.]
 [0. 0. 1. 1.]
 [0. 1. 0. 1.]
 [1. 1. 2. 0.]]


## Your answer to Question 3

The values of some cells are changed. Now only cell \[1, 2\] has the value 2.

Since the window size is changed, the contexts for word counting are also changed.

Now for the cell \[3, 2\] with the value 2, the row means word "cheese" and the column means word "and". Since now our window is 2, for word "cheese", it appears in the context "and crackers" and "wine and", so the co-occurence is 2. However, for the cell \[2, 3\] with the value 1, the row means word "and". The column means word "cheese". For word "and", it has the context "crackers" and "cheese". So the value is 1. 

## PPMI matrix

Now we'll create a PPMI matrix.  Please refer to the Jurafsky and Martin "Vector Semantics" chapter for more information.

In [13]:
def create_PPMI_matrix(term_context_matrix, k=0.01):
  '''Given a term context matrix, output a PPMI matrix.
  See section 15.1 in the textbook.
  Input:
    term_context_matrix: A nxn numpy array, where n is
        the numer of tokens in the vocab.
  Returns: A nxn numpy matrix, where A_ij is equal to the
     point-wise mutual information between the ith word
     and the jth word in the term_context_matrix.
  ''' 
  term_context_matrix += k # add constant k to each element to eliminate zeros (LaPlace smoothing)
  total_sum = np.sum(term_context_matrix)
  print("total sum: " + str(total_sum))
  Pij = term_context_matrix / total_sum
  Pi = np.sum(term_context_matrix,axis=1) / total_sum
  Pj = np.sum(term_context_matrix,axis=0) / total_sum
  PiTimesPj = np.outer(Pi, Pj)
  ppmi = np.where(np.log2(Pij / (PiTimesPj)) > 0, np.log2(Pij / (PiTimesPj)), 0)
    # keep only the ppmi elements greater than 0.
  return ppmi



Let's try it out.

In [14]:
print(create_PPMI_matrix(create_term_context_matrix(line_tuples_demo, vocab_demo, context_window_size=1)))

total sum: 8.16
[[0.         0.         0.97198562 0.        ]
 [0.         0.         0.97198562 0.        ]
 [0.97198562 0.97198562 0.         0.99284021]
 [0.         0.         0.99284021 0.        ]]


## Question 4

Please choose a cell of this PPMI matrix, identify which row/column you are referring to, and explain what the value in this cell means as well as how it is calculated.

## Your answer to Question 4

The cell \[2, 3\] has the largest value 0.99284021. Here, the row is word "cheese" and column is word "and". 

The value 0.99284021 is the Positive Pointwise Mutual Information. It represents whether a context word is particularly informative about the target word.

The calculation is based on the term-context matrix in question2. At first, we add a constant k = 0.01 to all the values in term-context martix _**M**_. Then, we get the sum _**S**_ of all the values in term-context matrix. Then we calculate _**Pij**_. _**Pij**_ is the value of term-context matrix _**M**_ divide the sum _**S**_. After that, we calculate the _**Pi**_ and _**Pj**_. _**Pi**_ is the array of sum of the values in each row. _**Pj**_ is the array of sum of the values in each column. Then, we calculate the outer product of  _**Pi**_ and _**Pj**_ as _**PiTimesPj**_. Finally, _**PPMI**_ is calculated by _**log2(Pij/PiTimesPj)**_. Here, we only keep the _**PPMI**_ whose value is larger than 0. If the value of _**PPMI**_ is smaller than 0, we set it as 0.

## Question 5

Now please re-run the code that generates a term-context matrix and a PPMI matrix, but this time using a context window of 2 rather than 1.  For each of the matrices that you generate (the new term-context matrix and the new PPMI matrix), please again choose a cell and explain what it means.

In [15]:
print(create_PPMI_matrix(create_term_context_matrix(line_tuples_demo, vocab_demo, context_window_size=2)))

total sum: 10.16
[[0.         0.         0.31625934 0.72654331]
 [0.         0.         0.31625934 0.72654331]
 [0.         1.30204549 0.         0.72654331]
 [1.28824497 0.31625934 0.32331341 0.        ]]


## Your answer to Question 5

I choose the cell \[2, 1\]. It has the largest value 1.30204549. The row is word "and". The column is word "crackers". It means that "and" and "cracker" co-occur more than they were independant.

## Question 6

The code used to create the PPMI matrix is "vectorized".  Please explain what that means.  Feel free to Google around.

If you can, please also write a for-loop which would calculate the value of each individual cell of the PPMI matrix.

## Your answer to Question 6

I think "vectorized" means that we calculate the PPMI based on vector and matrix by some linear algebra methods.

In [17]:
import math

def create_PPMI_matrix_for_a_cell(term_context_matrix, m, n, k=0.01):
    term_context_matrix += k # add constant k to each element to eliminate zeros (LaPlace smoothing)
    total_sum = 0
    for i in range(term_context_matrix.shape[0]):
        for j in range(term_context_matrix.shape[1]):
            total_sum += term_context_matrix[i, j]
    print("total sum: " + str(total_sum))
    Pij = term_context_matrix[m, n] / total_sum 
    
    Pi = 0
    for i in range(term_context_matrix.shape[1]):
        Pi += term_context_matrix[m, i]
    Pi = Pi / total_sum
    Pj = 0
    for i in range(term_context_matrix.shape[0]):
        Pj += term_context_matrix[i, n]
    Pj = Pj / total_sum
    PiTimesPj = Pi * Pj
   
    ppmi = math.log2(Pij / PiTimesPj) if math.log2(Pij / PiTimesPj) > 0 else 0
    
    return ppmi

print(create_PPMI_matrix_for_a_cell(create_term_context_matrix(line_tuples_demo, vocab_demo, context_window_size=1), 1, 2))

total sum: 8.159999999999998
0.9719856238304037


With my for-loop method which would calculate the value of each individual cell of the PPMI matrix, you just need to give the term-context matrix and the row index as well as the column index of the cell to the function, it can give you the value of PPMI for that cell.

## tf-idf matrix

Now we'll create a tf-idf matrix: a matrix indicating, for each word, its tf-idf among our collection of documents.


In [21]:
def create_tf_idf_matrix(term_document_matrix):
  '''Given the term document matrix, output a tf-idf weighted version.
  Input:
    term_document_matrix: Numpy array where each column represents a document 
    and each row, the frequency of a word in that document.

  Returns:
    A numpy array with the same dimension as term_document_matrix, where
    A_ij is weighted by the inverse document frequency of document h.
  '''
  N_docs = term_document_matrix.shape[1] # number of columns of the matrix = number of documents
  idf = np.count_nonzero(term_document_matrix, axis=1) # this calculates doc frequency of each term
  tfidf = (np.log(N_docs / idf) * term_document_matrix.T).T
  return tfidf

print("term document matrix")
print(create_term_document_matrix(line_tuples_demo, doc_names_demo, vocab_demo))
print("tf-idf matrix")
print(create_tf_idf_matrix((create_term_document_matrix(line_tuples_demo, doc_names_demo, vocab_demo))))

term document matrix
number of words =4
number of documents = 2
[[0. 1.]
 [1. 0.]
 [1. 1.]
 [1. 1.]]
tf-idf matrix
number of words =4
number of documents = 2
[[0.         0.69314718]
 [0.69314718 0.        ]
 [0.         0.        ]
 [0.         0.        ]]


## Question 7

Please choose a cell in this matrix and explain what the value in it means and how it is calculated.

## Your answer to Question 7

I choose the cell \[0, 1\] with the value 0.69314718. It means the frequency of word among the collection of documents. Also, it means how important a word is to a document in a collection or corpus. Since "cheese" and "and" appear in both two documents, so their value is 0.

To calculate the value, first, we calculate the tf-idf. Then, we multiply the log of tf-idf and the transpose of term-document matrix. Finally, we get the transpose of the result in last step.

Next, we write code to compute the cosine similarity between vectors.

In [19]:
from scipy import spatial
def compute_cosine_similarity(vector1, vector2):
  '''Computes the cosine similarity of the two input vectors.

  Inputs:
    vector1: A nx1 numpy array
    vector2: A nx1 numpy array

  Returns:
    A scalar similarity value.
  '''
  
  return 1 - spatial.distance.cosine(vector1, vector2)

print(compute_cosine_similarity([1,2,3], [3,2,1]))
print(compute_cosine_similarity([1,2,3], [10,20,30]))
print(compute_cosine_similarity([1,2,3], [4,5,6]))
print(compute_cosine_similarity([1,2,3], [3,30,300]))

0.7142857142857143
1.0
0.9746318461970761
0.8536086925965384


## Question 8

What is cosine similarity?   What is the range of possible values that it can yield?  (Feel free to read online about this to come up with your answer.)  Can you explain the outputs printed above on an intuitive or conceptual level?  What does a "higher" number mean?  What does a "lower" number mean?

## Your anwer to Question 8

Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. The range of possible values is from -1 to 1. 

The outputs printed above show the similarity of each pair of vectors. Higher number means that the two vectors are more similar. Lower number means two vectors are more different.

Next, we will write a function which ranks the similarity of all documents to a single target document.   In the spirit of the Honor Code, I acknowledge that some of this code is from Stack Exchange.

In [29]:
from scipy.spatial import distance


def rank_docs(target_doc_index, term_document_matrix):
  ''' Ranks the similarity of all of the plays to the target play.
  Inputs:
    target_doc_index: The integer index of the doc we want to compare all others against.
    term_document_matrix: The term-document matrix as a mxn numpy array.

  Returns:
    The integer index of the doc with the least (cosine) distance to the doc indexed by target_doc_index.
  '''

  target = term_document_matrix.T[target_doc_index]
 # print("target")
 # print(target) you can uncomment these if you want to see it
  vectors = list(term_document_matrix.T)
 # print("vectors")
 # print(vectors)
  distances = distance.cdist([target], vectors, "cosine")[0]
  min_index = np.argpartition(distances, 1)[1] # this gets the SECOND-closest vector
                # because the CLOSEST vector is always the target vector itself.

  return min_index



term_doc_matrix = create_term_document_matrix(line_tuples_demo, doc_names_demo, vocab_demo)
print(term_doc_matrix)

print("Index of the document most similar document to Doc1 is.... ")
print(rank_docs(1, term_doc_matrix))

number of words =4
number of documents = 2
[[0. 1.]
 [1. 0.]
 [1. 1.]
 [1. 1.]]
Index of the document most similar document to Doc1 is.... 
0


## Question 9

What happened here?  What does "0" mean?

## Your answer to Question 9

It calculates the second closest document for the doc1 based on the consine similarity. Here, the first closest document is always the target document itself. 
0 means that the closest document to doc1 is doc0 except doc1 itself.

Now, we'll write code to try to find the most similar WORD to another word, based on the term-context-matrix.

In [39]:


def rank_words(target_word_index, term_context_matrix):
  ''' Ranks the similarity of all of the words to the target word.

  Inputs:
    target_word_index: The index of the word we want to compare all others against.
    term_context_matrix: Numpy matrix where the ith row represents a vector embedding of the ith word.

  Returns:
    A length-n list of integer word indices, ordered by decreasing (cosine) similarity to the 
    target word indexed by word_index
  '''

  target = term_context_matrix.T[target_word_index]
  print("target")
  print(target) #you can uncomment these if you want to see it
  vectors = list(term_context_matrix.T)
  print("vectors")
  print(vectors)
  distances = distance.cdist([target], vectors, "cosine")[0]
  min_index = np.argpartition(distances, 1)[1] # this gets the SECOND-closest vector
                # because the CLOSEST vector is always the target vector itself.

  return min_index



term_context_matrix = create_term_context_matrix(line_tuples_demo, vocab_demo, 1)
print(term_context_matrix)

print("Index of the word most similar to Word3 is.... ")
print(rank_words(3, term_context_matrix))


[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 1. 0. 2.]
 [0. 0. 2. 0.]]
Index of the word most similar to Word3 is.... 
target
[0. 0. 2. 0.]
vectors
[array([0., 0., 1., 0.]), array([0., 0., 1., 0.]), array([1., 1., 0., 2.]), array([0., 0., 2., 0.])]
1


## Question 10

What does this mean?  What is Word3?  What is Word1?  Why is Word1 similar to Word3?  (How does all this relate to cheese, crackers, and wine?)

## Your answer to Question 10

It calculates the closest word to the target word except itself. 
Word3 is "cheese".
Word1 is "crackers".
Since the consine similarity of vector for "cheese" appear in context and vector for "crackers" is biggest, word1 is similar to word3.

##  Question 11

Why is it useful to run our code on a tiny corpus instead of a much bigger one?  When you do big data projects, do you also practice and debug on a tiny subset of the data?  Why or why not?

Also -- this homework uses Numpy arrays (mainly because those were used in the source material that this is adapted from), but it would actually be better to use Pandas dataframes where it is easier to label rows and columns.  If you are feeling ambitious, can you make a Pandas array of our term-by-document matrix (for doc1="cheese and crackers", doc2= "wine and cheese") with the rows and columns labeled in an easy-to-read manner?

## Your answer to Question 11

I think it can save more time and easy to debug on the tiny corpus than much bigger one.
I also practice and debug on a tiny subset of the data to develop the new functions. It's better for me to understand the algorithms and solve the bugs. If my function can work on the smaller dataset, it can work on a bigger dataset, too.

In [42]:
import pandas as pd

term_document_matrix = create_term_document_matrix(line_tuples_demo, doc_names_demo, vocab_demo)
term_document_dataframe = pd.DataFrame(data=term_document_matrix, 
                                       index=['wine', 'crackers', 'and', 'cheese'], 
                                       columns=['cheese_and_crackers', 'wine_and_cheese'])
print(term_document_dataframe)

number of words =4
number of documents = 2
          cheese_and_crackers  wine_and_cheese
wine                      0.0              1.0
crackers                  1.0              0.0
and                       1.0              1.0
cheese                    1.0              1.0


## Question 12

Please read this article: https://www.quantamagazine.org/machines-beat-humans-on-a-reading-test-but-do-they-understand-20191017 and then indicate FIVE substantive facts that you learned.

## Your answer to Question 12

1. Traditional machine learning models cannot solve GLUE since they cannot learn anything substantial about language itself. However, deep learning model like BERT can solve GLUE to some extent and has good performance.
2. A neural network pretrained with word embeddings is still blind to the meaning of words at the sentence level. For example,‘a man bit the dog’ and ‘a dog bit the man’ are exactly the same thing. Then the researches tried to equip the network with richer rulebooks — not just for vocabulary, but for syntax and context as well. They proposed the language modeling and with fine-tuning, it has the great performance on the GLUE.
3. The three ingredients of BERT are a deep pretrained language model, attention and bidirectionality. They existed independently before BERT. However, the scientists in Google combined them together in such a powerful way and created the innovative BERT.
4. BERT’s “pie crust” incorporates a number of structural design decisions that affect how well it works. These include the size of the neural network being baked, the amount of pretraining data, how that pretraining data is masked and how long the neural network gets to train on it. So there are still many researchers tweaking these design decisions to make their BERT models work better. 
5. BERT’s high performance on certain GLUE tasks might be attributed to spurious cues in the training data for those tasks. One way to encourage progress toward robust understanding is to focus not just on building a better BERT, but also on designing better benchmarks and training data that lower the possibility of Clever Hans–style cheating.

## Question 13

Please try out AllenNLP's "textual entailment" tool: https://demo.allennlp.org/textual-entailment

How do you think this tool was built?  How good is it?  What types of sentence pairs can it handle well, what pairs does it struggle with, and why do you think that is?

## Your answer to Question 13

Textual entailment is the task of determining whether two natural language sentences are (i) contradicting each other, (ii) not related, or whether (iii) the first sentence (called premise) entails the second sentence (called hypothesis). Via the paper, I learned that the textual entailment uses (i) conditional encoding via two LSTM, one over the premise and one over the hypothesis conditioned on the representation of the premise (ii) attention only based on the last output vector and (iii) word-by-word attention based on the output vectors of the hypothesis.

The performance is good. I tried all the sample pairs of premise and hypothesis. The text entailment tool gave the correct answers.

For the pairs with common words like "Two women are wandering along the shore drinking iced tea." and "Two women are sitting on a blanket near some rocks talking about politics.", it handles well.

For the pairs without the common words like "I am tired" and "I am going to bed". It struggles and doesn't give the correct answer based on human understanding.

I think this model still depends on the vocabulary and syntax of sentences a lot. So for those pairs without common words, the model cannot catch the similarity of words and perform poorly.


## Question 14

What do you think you will do for your final project in this class?  Please write a few sentences so that I can give you some feedback.

## Your answer to Question 14

For my final project, I think that spell checker is really interesting and useful in our daily life. I want to improve the Norvig’s spell-checker by considering the candidate words of more edit distane, 2-gram or 3-gram for catching more complex multi-typos, hamming distance and word-frequency model based chooser for suggestions and so on. Also, I want to explore some machine learning methods to check the grammer errors in the sentence. Finally, I will try to build the user interface for my spell checker to show it in class.